## 셀리니움 활용해서 sparkkorea.com quiz에 접근을 한다
### 이후 spark 관련 퀴즈 타이틀과 링크정보를 수집하여 데이터프레임으로 변환한다

In [4]:
from selenium import webdriver

import pandas as pd

options = webdriver.ChromeOptions()

# 헤드리스 옵션 / 일반 옵션
# 일반 - 브라우저를 진짜 띄운다.
# 헤드리스 - 브라우저를 띄우지 않고 크롤링.

options.add_argument("window-size=1920*1080") #pypi.org 들가면 나와있따.

driverLoc = "../addon/chromedriver/chromedriver.exe"

driver = webdriver.Chrome(executable_path=driverLoc, options=options) # 파라미터를 지정(변수로 지정해주면) 했으면 둘다 변수로 지정해줘야한다. 

driver.implicitly_wait(3)

targetUrl = "https://www.sparkkorea.com"

## get 함수를 써서 url정보를 던져주면 이동.
driver.get(targetUrl)

## 퀴즈버튼 Xpath 저장.
quizBtnXpath = '//*[@id="menu-item-382"]/a' 

## 변수에 퀴즈버튼 Xapth 넣어주기
quizBtn = driver.find_element_by_xpath(quizBtnXpath)

## 퀴즈버튼 클릭.
quizBtn.click()

## 클릭후 이동한 퀴즈 창의 페이지 소스 추출
quizSrc = driver.page_source

### 인제 bs4의 영역이다. 크롤링 해보자.

import bs4

bsObj = bs4.BeautifulSoup(quizSrc, "html.parser")

## attrs = {"속성명":"속성이름"} 부분태그로 접근
targetList = bsObj.find(name="div",attrs = {"id":"id_spark_quiz"})



## a태그 추출
atagList = targetList.findAll("a")

columnList = []
rowList = []

for i in range(0, len(atagList)):
    

    eachTag = atagList[i]

    eachName = eachTag.text
    
    eachLink = eachTag.attrs["href"] ##속성이 href인것!
    columnList.append(eachName)
    columnList.append(eachLink)
    rowList.append(columnList)
    columnList = []

    



columnName = {"퀴즈명", "링크"}

result = pd.DataFrame(rowList)

result.columns = columnName

result

,퀴즈명,링크
0,6/13 Spark 심화과정,https://forms.gle/Fw49w9GhWQChDcZm7
1,6/13 Spark 기본과정,https://forms.gle/G4TcXm3fKuHLHA6D6
2,6/18 Spark [MAP_FILTER],https://forms.gle/M8gr1kC2ubA3UDVp8
3,6/18 Spark GroupBy 심화,https://forms.gle/h8w5mZ4MNaPLCPbi6
4,6/25 Spark RDD 실전 분석,https://forms.gle/q5yL6QHfueDLM5w27
5,6/27 Spark RDD 실전 분석2,https://forms.gle/Gxb4y6LfVYiaLu4M7


In [ ]:
#3 데이터프레임을 csv 파일로 저장하기.

In [2]:
result.to_csv("./quiz_result.csv", index=False, encoding = "ms949")